# Game Recommendation System - Project Documentation

## 1. Project Overview
**YourNextGame** is an intelligent, single-page web application designed to help users discover video games based on their current mood and demographic profile. It combines aesthetic modern UI design with a hybrid recommendation engine that runs entirely on the client side, powered by weights learned from a synthetic dataset.

### Key Objectives:
- **Personalization**: Tailor suggestions based on Age, Gender, and Region.
- **Mood-Based Discovery**: specific tags like "Chill", "Competitive", "Story-rich".
- **Interactive UI**: Glassmorphism design, 3D backgrounds, and dynamic animations.
- **Data Persistence**: Saves user profile and favorites locally in the browser.

## 2. User Interface & Experience Features

### 2.1 Hero & Navigation
- **3D Background**: Utilizes `Three.js` to render a dynamic starfield effect that reacts to mouse movement, creating an immersive landing experience.
- **Glassmorphism Header**: A translucent navigation bar with blur effects (`backdrop-filter: blur`) for a modern look.
- **Live Stats**: Displays real-time counts of active users and games in the library.

### 2.2 Game Cards
Each game is displayed as an interactive card with rich functionality:
- **Smart Image Loading**: 
  - Uses high-quality Steam Assets where available.
  - Fallback: If an image fails to load, a **Gradient Title Card** is automatically generated using CSS linear gradients and the game title.
- **Quick Actions**:
  - **▶ Trailer**: Performs a targeted YouTube search for the game's gameplay trailer in a new tab.
  - **⬇️ Download**: Smart-links to the Steam Store for PC games or a Google Shopping search for console exclusives.
  - **❤️ Favorite**: Toggles the game in the user's local favorites list.

### 2.3 Personalization Modal (AI Calibration)
- **Visuals**: Triggered by a "✨ Customize AI" button with a pulsing glow animation.
- **Inputs**: Collects User Age, Gender, and Region.
- **Feedback**: A bouncing "!" badge alerts users to set up their profile if they haven't done so.

## 3. The Recommendation Engine (Machine Learning)

The core of the system is a **Hybrid Recommendation Model** that combines rule-based filtering (Moods) with a probabilistic demographic model (ML Weights).

### 3.1 The Inference Engine (JavaScript)
Runs continuously in the browser (`MLRecommendationEngine` class).

**Algorithm:**
$$ Score = BaseScore(Mood) + MLBoost(Profile) $$

1. **Base Score (Content-Based)**: 
   - +2.0 points for exact mood match.
   - +1.0 point for related genre match (e.g., "Action" mood matches "FPS" genre).
   - +0.5 points for "Chill" mood matching newer games (> 2018).

2. **ML Boost (Demographic Filtering)**:
   - The engine looks up a pre-calculated weight table based on the user's `Age Group`, `Gender`, and `Region`.
   - *Example*: A "Mature" user gets a +1.75 boost for "Strategy" games.
   - *Example*: An "ASIA" region user gets a +1.25 boost for "JRPG" titles.

### 3.2 Training Pipeline

Since we don't have a massive backend, we trained the weights offline using a Python pipeline and injected them into the Javascript engine.

#### Step A: Synthetic Data Generation
We created a script to simulate 5,000 users with biases (e.g., Younger users preferring competitive games).

In [ ]:
import csv
import random

# snippet from generate_dataset.py
def generate_preference(age, gender, region, genre):
    score = 3.0 # Neutral
    noise = random.uniform(-0.5, 0.5)

    # Encoded Logic Pattern (The "Ground Truth")
    if age < 18:
        if genre in ['FPS', 'Battle Royale']: score += 1.5
    elif region == 'ASIA':
        if genre == 'JRPG': score += 1.2
    
    return max(1, min(5, int(round(score + noise))))

#### Step B: Weight Training
We processed the CSV to calculate the "Affinity Weight" for each demographic bucket. The weight is calculated as the deviation from the neutral score (3.0).

$$ Weight_{Genre} = AverageRating_{Genre} - 3.0 $$

In [ ]:
import csv
from collections import defaultdict

# snippet from train_model.py
data_points = defaultdict(list)
NEUTRAL_SCORE = 3.0

# ... reading CSV ...
# for row in reader:
#    data_points[('age', 'young', 'FPS')].append(rating)

# Calculate Weights
learned_weights = {}
for key, ratings in data_points.items():
    avg = sum(ratings) / len(ratings)
    weight = avg - NEUTRAL_SCORE
    learned_weights[key] = round(weight, 2)

## 4. User Profile System

The application features a robust profile system that serves as the user's dashboard.

1.  **Identity Management**:
    -   Users can edit their **Username** directly in the UI.
    -   Persists to `localStorage`, welcoming them back by name.

2.  **Gamer Personality Summary**:
    -   **Dynamic Analysis**: The system analyzes the user's `Favorites` list in real-time.
    -   **Natural Language Generation**: It constructs a profile summary sentence.
    -   *Logic*: `"You are a true fan of {TopGenre} and {SecondGenre}. You seek {TopMood} experiences."`

3.  **Visualization**:
    -   Displays user Level and Favorites count.
    -   Shows a grid of all favorited games for quick access.

## 5. Technical Stack

| Component | Technology |
|-----------|------------|
| **Frontend** | HTML5, CSS3 (Variables, Flexbox, Grid), Vanilla JavaScript |
| **Graphics** | Three.js (WebGL Background) |
| **State Management** | LocalStorage API (Persistence) |
| **Data Processing** | Python (Pandas/CSV for offline training) |
| **Icons/Assets** | SVG, Steam CDN, Unsplash |